In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)
import timeit

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes
# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Tue 04 Apr 2023 03:44:33 PM PDT
ndbs
Linux gen-slurm-sexec-p0001 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/big_data_tests


# Find data

Run of 18 seeds, reflecting a population of 18 million, from Rajan on 3/17/2023:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19
```

In [2]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/v1.2_parallelize_business_names'
    '/united_states_of_america/2023_03_16_17_21_09'
    '/final_results/2023_03_17_15_28_19'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 44K
drwxrwsr-x  5 rmudambi IHME-Simulationscience 2.0K Mar 17 16:06 ..
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 16:04 tax_dependents_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:59 tax_1040_observer
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Mar 17 15:53 .
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:51 tax_w2_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:50 social_security_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 wic_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_cps
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_acs
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:48 decennial_census_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience  18K Mar 17 15:29 logs


In [3]:
!ls -halt $output_dir/tax_w2_observer

total 12G
-rw-r--r--  1 rmudambi IHME-Simulationscience 669M Mar 17 15:58 tax_w2_observer_6817.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 665M Mar 17 15:58 tax_w2_observer_4344.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 670M Mar 17 15:58 tax_w2_observer_3541.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 679M Mar 17 15:57 tax_w2_observer_4507.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 674M Mar 17 15:57 tax_w2_observer_6545.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_781.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 677M Mar 17 15:57 tax_w2_observer_9672.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_8869.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 675M Mar 17 15:57 tax_w2_observer_6810.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_1362.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 682M Mar 17 15:

# Check how much space full data takes up

24 GiB = 25.77 GB

In [4]:
# Total of 24 GiB in all subdirectories
!du -sh $output_dir

24G	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [5]:
# 24 GiB = 25.77 GB
2**30/1e9 * 24

25.769803776000003

# Load files using Categoricals, then save as HDF, time the steps, and see total disk usage

In [6]:
save_dir = '/share/scratch/users/ndbs/prl/2023_03_16_17_21_09'
!ls -halt $save_dir

total 8.0K
drwxrwxr-x 2 ndbs Domain Users    0 Apr  4 15:44 .
drwxrwxr-x 6 ndbs Domain Users 2.5K Apr  4 15:26 ..


In [7]:
%%time
observer_names = []
nrows = None
ext = '.csv.bz2'
seed_length=4
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    print("="*50)
    print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
#             seed = filename_root[:-seed_length]
            filepath = os.path.join(dirpath, filename)
            %time shards.append(datatypes.load_csv_data(filepath, convert_str_ids=True, nrows=nrows))
    if shards:
        print("\nSetting categories for", observer_name)
        categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
        for col in categorical_cols:
            categories = shards[0][col].cat.categories
            for shard in shards[1:]:
                categories = categories.union(shard[col].cat.categories)
            for shard in shards:
                shard[col] = shard[col].cat.set_categories(categories)
        print("Concatenating data for", observer_name)
        %time df = pd.concat(shards, ignore_index=True)
        print("Calculating memory usage")
        %time print(sizemb(df), "MB")
        observer_names.append(observer_name)
        print("Saving data for", observer_name)
        %time df.to_hdf( \
            f'{save_dir}/{observer_name}.hdf', key=observer_name, \
            format='table', complib='bzip2', complevel=9, \
        )
        # Not sure if I need this, but we can delete dataframes to save memory
        del df
        del shards
print('\nObservers:', observer_names, '\n')
!ls -halt $save_dir

Loading data from 2023_03_17_15_28_19 

Loading data from decennial_census_observer 

CPU times: user 38 s, sys: 1.21 s, total: 39.2 s
Wall time: 39.3 s
CPU times: user 40 s, sys: 947 ms, total: 40.9 s
Wall time: 41 s
CPU times: user 39.9 s, sys: 811 ms, total: 40.7 s
Wall time: 40.8 s
CPU times: user 41 s, sys: 723 ms, total: 41.7 s
Wall time: 41.7 s
CPU times: user 41 s, sys: 668 ms, total: 41.6 s
Wall time: 41.7 s
CPU times: user 42.1 s, sys: 712 ms, total: 42.9 s
Wall time: 42.9 s
CPU times: user 42.6 s, sys: 696 ms, total: 43.3 s
Wall time: 43.4 s
CPU times: user 42.6 s, sys: 704 ms, total: 43.3 s
Wall time: 43.3 s
CPU times: user 43 s, sys: 645 ms, total: 43.7 s
Wall time: 43.7 s
CPU times: user 43.3 s, sys: 656 ms, total: 43.9 s
Wall time: 44 s
CPU times: user 42.1 s, sys: 792 ms, total: 42.9 s
Wall time: 43 s
CPU times: user 44.7 s, sys: 587 ms, total: 45.2 s
Wall time: 45.3 s
CPU times: user 44.7 s, sys: 680 ms, total: 45.3 s
Wall time: 45.4 s
CPU times: user 44.6 s, sys: 552 

CPU times: user 2.17 s, sys: 8.08 ms, total: 2.18 s
Wall time: 2.19 s
CPU times: user 1.88 s, sys: 7.92 ms, total: 1.89 s
Wall time: 1.9 s
CPU times: user 2.3 s, sys: 8 ms, total: 2.3 s
Wall time: 2.31 s
CPU times: user 1.92 s, sys: 15.9 ms, total: 1.93 s
Wall time: 1.94 s
CPU times: user 1.94 s, sys: 8.02 ms, total: 1.95 s
Wall time: 1.96 s
CPU times: user 2.1 s, sys: 3.94 ms, total: 2.1 s
Wall time: 2.11 s
CPU times: user 2.19 s, sys: 0 ns, total: 2.19 s
Wall time: 2.2 s
CPU times: user 2.65 s, sys: 3.91 ms, total: 2.66 s
Wall time: 2.67 s
CPU times: user 2.07 s, sys: 3.93 ms, total: 2.07 s
Wall time: 2.08 s
CPU times: user 2.44 s, sys: 4.01 ms, total: 2.44 s
Wall time: 2.45 s
CPU times: user 2.1 s, sys: 3.81 ms, total: 2.11 s
Wall time: 2.12 s
CPU times: user 1.9 s, sys: 28 µs, total: 1.9 s
Wall time: 1.9 s
CPU times: user 2 s, sys: 4.1 ms, total: 2 s
Wall time: 2.01 s
CPU times: user 2.1 s, sys: 12 ms, total: 2.11 s
Wall time: 2.12 s

Setting categories and concatenating data for  

# Read data back in, time it, and see how much space everything takes up

In [8]:
%%time
dfs = {}
ext = '.hdf'
for dirpath, dirnames, filenames in os.walk(save_dir):
    print(dirpath, dirnames, filenames)
    for filename in filenames:
        if filename.endswith(ext):
            observer_name = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            print("\nLoading data from", observer_name)
            %time dfs[observer_name] = pd.read_hdf(filepath)
            print("Computing memory usage")
            %time print(sizemb(dfs[observer_name]), 'MB')

dfs = MappingViaAttributes(dfs)
print(dfs.keylist())
print("\nTotal memory usage")
%time print(sum(sizemb(df) for df in dfs.values()), "MB")
print("\nTotal time")

/share/scratch/users/ndbs/prl/2023_03_16_17_21_09 [] ['tax_1040_observer.hdf', 'wic_observer.hdf', 'tax_dependents_observer.hdf', 'decennial_census_observer.hdf', 'tax_w2_observer.hdf', 'social_security_observer.hdf', 'household_survey_observer_acs.hdf', 'household_survey_observer_cps.hdf']

Loading data from tax_1040_observer
CPU times: user 7min 43s, sys: 12.3 s, total: 7min 55s
Wall time: 8min 3s
Computing memory usage
13370.361975 MB
CPU times: user 311 ms, sys: 60 ms, total: 371 ms
Wall time: 370 ms

Loading data from wic_observer
CPU times: user 7.35 s, sys: 96.4 ms, total: 7.45 s
Wall time: 7.54 s
Computing memory usage
293.823706 MB
CPU times: user 89.7 ms, sys: 32 µs, total: 89.7 ms
Wall time: 89.6 ms

Loading data from tax_dependents_observer
CPU times: user 2min 32s, sys: 3.7 s, total: 2min 35s
Wall time: 2min 38s
Computing memory usage
4655.984061 MB
CPU times: user 196 ms, sys: 28 ms, total: 224 ms
Wall time: 224 ms

Loading data from decennial_census_observer
CPU times: u